In [31]:
import pandas as pd
import datetime as dt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
events = pd.read_csv('../fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')

/home/aaron/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
train_data = pd.read_csv('../fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [26]:
test_kaggle = pd.read_csv('../fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

In [7]:
events['week'] = pd.to_datetime(events['timestamp']).dt.week

In [8]:
events['week'].value_counts()

21    540477
20    495049
22    408822
19    166141
18    118139
17     96538
16     67727
15     65513
14     60947
13     55405
12     55377
11     40157
10     31867
9      28420
8      21058
5      16097
7      15546
6      15323
3      12688
2      12122
4      11537
1       6731
Name: week, dtype: int64

In [9]:
#SEPARO EN PERIODOS DE DOS SEMANAS
def periodo (week):
    return (week+1)//2

events['periodo'] = events['week'].apply(lambda week: periodo(week))

In [10]:
events['periodo'].value_counts()

11    949299
10    661190
9     214677
8     133240
7     116352
6      95534
5      60287
4      36604
3      31420
2      24225
1      18853
Name: periodo, dtype: int64

# Feature: Cantidad de eventos por usuario 

Solo trabajo con alguno de los periodos

In [11]:
eventos = events.loc[events['periodo']>6]

In [12]:
eventos= eventos[['person','periodo','timestamp']]

In [13]:
eventos_por_periodo = eventos.groupby(['periodo','person']).count().unstack(level=0)

In [14]:
eventos_por_periodo.reset_index(inplace=True)

In [15]:
eventos_por_periodo.columns = [col[1] for col in eventos_por_periodo.columns]

In [16]:
eventos_por_periodo.rename(columns={'':'person'},inplace=True)
eventos_por_periodo.fillna(value=0,inplace=True)

In [24]:
eventos_por_periodo.rename(columns={7:'eventos_periodo_7',8:'eventos_periodo_8',9:'eventos_periodo_9',10:'eventos_periodo_10',11:'eventos_periodo_11'}, inplace=True)

In [27]:
train_data = pd.merge(train_data,eventos_por_periodo,on='person',how='left')
test_kaggle = pd.merge(test_kaggle,eventos_por_periodo,on='person',how='left')

In [44]:
test_kaggle.fillna(value=0,inplace=True)

# Gradient boosting

In [22]:
gBoosting = GradientBoostingClassifier(n_estimators=20, max_features=4,max_depth=10)

In [41]:
X = train_data[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']]
Y = train_data['label']

X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [42]:
gBoosting.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=4, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=20,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [43]:
gBoosting.score(X_test,Y_test)

0.9459227467811159

In [45]:
test_kaggle['prediction'] = gBoosting.predict(test_kaggle[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']])

In [ ]:
test

In [46]:
submit_kaggle = pd.DataFrame({'person':test_kaggle['person'],'label':test_kaggle['prediction']})
submit_kaggle.reset_index(inplace = True)
submit_kaggle.drop(columns='index',inplace=True)
submit_kaggle.set_index(keys='person',inplace=True)

In [47]:
submit_kaggle.to_csv('submit_kaggle.csv')

# Feature: region - resolucion de pantalla

In [35]:
region = events[['person','region','screen_resolution']].dropna()
region

,person,region,screen_resolution
2136629,4640420b,Parana,320x570
2136630,4640420b,Parana,320x570
2136631,4640420b,Unknown,1280x1024
2136632,4640420b,Parana,800x1280
2136633,4640420b,Parana,800x1280
2136634,4640420b,Parana,320x570
2136635,4640420b,Parana,800x1280
2136636,4640420b,Sao Paulo,320x570
2136637,4640420b,Unknown,1280x1024
2136638,4640420b,Parana,800x1280


In [36]:
region['region_resolucion'] = region['region']+region['screen_resolution']
region_resolucion = region[['person','region_resolucion']]
region_resolucion

,person,region_resolucion
2136629,4640420b,Parana320x570
2136630,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136633,4640420b,Parana800x1280
2136634,4640420b,Parana320x570
2136635,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136637,4640420b,Unknown1280x1024
2136638,4640420b,Parana800x1280


In [37]:
region_resolucion = region_resolucion.drop_duplicates()
region_resolucion

,person,region_resolucion
2136629,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136640,4640420b,Parana320x522
2136652,4640420b,Santa Catarina1280x1024
2136655,4640420b,Santa Catarina320x522
2136659,4640420b,Sao Paulo1280x1024
2136660,f35dddc8,Piaui360x640
2136661,c82ecb12,Rio de Janeiro360x640


In [38]:
region_resolucion['region_resolucion_h'] = region_resolucion['region_resolucion'].apply(lambda a: hash(a))
region_resolucion

/home/euge/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,person,region_resolucion,region_resolucion_h
2136629,4640420b,Parana320x570,-3486592093772705005
2136631,4640420b,Unknown1280x1024,-5735015092186701903
2136632,4640420b,Parana800x1280,-4233600664636165113
2136636,4640420b,Sao Paulo320x570,-8836238896939786766
2136640,4640420b,Parana320x522,7068915030938652175
2136652,4640420b,Santa Catarina1280x1024,1630208838237318951
2136655,4640420b,Santa Catarina320x522,-4893498236986548312
2136659,4640420b,Sao Paulo1280x1024,-7074889735127548929
2136660,f35dddc8,Piaui360x640,4736145904860099221
2136661,c82ecb12,Rio de Janeiro360x640,1844095417007047287


In [39]:
train_data = train_data.merge(region_resolucion,on='person',how='left')

In [ ]:
train_data.to_csv('features/train_data.csv')

In [ ]:
test_kaggle = test_kaggle.merge(region_resolucion,on='person',how='left')

In [ ]:
test_kaggle.to_csv('test_kaggle.csv')